In [ ]:
%pip install transformers


In [ ]:
MODEL_NAME = "sentence-transformers/bert-base-nli-mean-tokens"

    
sentences = ["the fsb. a national savings bank. established march, 1865. chartered by the government of the united states. banking house 1507 pennsylvania avenue, opposite the treasury. deposits of five cents or any larger amounts received. six per cent interest paid on sums of five dollars or more. all deposits payable on demand, with interest due. all accounts strictly private and confidential. principal office, washington, d. c. branch offices in all the larger cities of the south and southwest. this great national savings institution, established by the authority of the united states government for the benefit of the freedmen, knows no distinction of race or color, and offers its great advantages to all classes alike. save the small sums. cut off your vices - don't smoke - don't drink - don't buy lottery tickets. put the money you save into the fsb. open from 9 a. m. to  4 p. m. each day, and on wednesday and saturday nights, to receive deposits only, from 6 1\/2 to 8 o'clock.",\
    "avoid buying costly garments",
    "spend not your hard-earned wages for filthy tobacco and useless drinks",
    "do not waste money at circuses, expensive pic-nics and excursions.",
    "cut off your vices - don't smoke - don't drink - don't buy lottery tickets.it is your duty to provide for your settlement in life"]


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

In [ ]:
tokens = {'input_ids': [], 'attention_mask': []}

In [ ]:
for sentence in sentences:
    new_tokens = tokenizer.encode_plus(sentence, max_length=128, truncation=True, padding='max_length', return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

In [ ]:
#Tokenizing, stacking all the input_ids and attention_masks into one tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [ ]:
outputs = model(**tokens)

In [ ]:
embeddings = outputs.last_hidden_state

In [ ]:
attention = tokens['attention_mask']
mask = attention.unsqueeze(-1).expand(embeddings.shape).float()
mask_embeddings = embeddings * mask

Figure out what mean pooling stuff below means and does

In [ ]:
summed = torch.sum(mask_embeddings, 1)

In [ ]:
counts = torch.clamp(mask.sum(1), min=1e-9)

In [ ]:
mean_pooled = summed / counts

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
mean_pooled = mean_pooled.detach().numpy()

cosine_similarity([mean_pooled[0]],mean_pooled[1:])